In [1]:
import pandas as pd
import numpy as np
import allel
import os

In [2]:
strain=['A_J','AKR_J','BALB_CJ','CBA_J','C3H_HeJ','DBA_2J','LP_J']

tools=['BioGraph*',
'breakdancer',
'clever',
'delly',
'gasv',
'gridss',
'indelminer',
'lumpexpress',
'mistrvar',
'pindel',
'popdel',
'rdxplorer',
'smoove',
'sniffles',
'crest',
'genomestrip',
'manta_diploidSV',
'Tardis']

In [3]:
df_10=pd.read_csv('../Data/analysis_files/df_10.csv')
df_100=pd.read_csv('../Data/analysis_files/df_100.csv')
df_1000=pd.read_csv('../Data/analysis_files/df_1000.csv')
df_10000=pd.read_csv('../Data/analysis_files/df_10000.csv')

df_10_nondel=pd.read_csv('../Data/analysis_files/df_10_nondel.csv')
df_100_nondel=pd.read_csv('../Data/analysis_files/df_100_nondel.csv')
df_1000_nondel=pd.read_csv('../Data/analysis_files/df_1000_nondel.csv')
df_10000_nondel=pd.read_csv('../Data/analysis_files/df_10000_nondel.csv')

pdList = [df_10,df_100,df_1000,df_10000,df_10_nondel,df_100_nondel,df_1000_nondel,df_10000_nondel] 
df_cat = pd.concat(pdList)

gs_=pd.read_csv('../Data/gold_standard/gs_')

In [4]:
df_cat.head()
df_cat = df_cat[df_cat['length'] >= 50]

In [5]:
df_cat.loc[(df_cat['length'] >= 50) & (df_cat['length'] < 100),'len'] = '50-100'
df_cat.loc[(df_cat['length'] >= 100) & (df_cat['length'] < 500),'len'] = '100-500'
df_cat.loc[(df_cat['length'] >= 500) & (df_cat['length'] < 1000),'len'] = '500-1000'
df_cat.loc[(df_cat['length'] >= 1000),'len'] = '1000+'
df_cat

,Unnamed: 0,tool,strain,length,flag,position,threshold,len
0,0,Tardis,LP_J,389,FP,3112809,10,100-500
1,1,Tardis,LP_J,1705,FP,6055228,10,1000+
2,2,Tardis,LP_J,2445,FP,6055227,10,1000+
3,3,Tardis,LP_J,582,FP,6057082,10,500-1000
4,4,Tardis,LP_J,140,FP,6829728,10,100-500
...,...,...,...,...,...,...,...,...
103923,103923,BioGraph*,A_J,36477,TN,61132293,10000,1000+
103924,103924,BioGraph*,A_J,36477,TN,61132293,10000,1000+
103925,103925,BioGraph*,A_J,8527,TN,61175238,10000,1000+
103926,103926,BioGraph*,A_J,8206,TN,61184017,10000,1000+


In [6]:
df_merge_all_len=pd.merge(df_cat, gs_,on=['strain','len'])
del df_merge_all_len['Unnamed: 0_x']
del df_merge_all_len['Unnamed: 0_y']
df_merge_all_len

,tool,strain,length,flag,position,threshold,len,n_true
0,Tardis,LP_J,389,FP,3112809,10,100-500,228
1,Tardis,LP_J,140,FP,6829728,10,100-500,228
2,Tardis,LP_J,196,FP,16852956,10,100-500,228
3,Tardis,LP_J,498,FP,17145726,10,100-500,228
4,Tardis,LP_J,231,FP,17246886,10,100-500,228
...,...,...,...,...,...,...,...,...
1063975,gasv,A_J,81,TN,37798574,10000,50-100,37
1063976,gasv,A_J,70,TN,41770354,10000,50-100,37
1063977,gasv,A_J,93,TN,42100244,10000,50-100,37
1063978,gasv,A_J,93,TN,42100244,10000,50-100,37


In [7]:
df_count_TP = df_merge_all_len[df_merge_all_len['flag'] == 'TP'].groupby(['tool','threshold','strain','len','n_true'],as_index=False)['flag'].count()
df_count_TP = df_count_TP.rename(columns={"flag": "nTP"})
df_count_TP.shape
df_count_TP.head()


,tool,threshold,strain,len,n_true,nTP
0,BioGraph*,10,AKR_J,100-500,234,19
1,BioGraph*,10,AKR_J,1000+,140,10
2,BioGraph*,10,AKR_J,50-100,37,2
3,BioGraph*,10,AKR_J,500-1000,83,5
4,BioGraph*,10,A_J,100-500,249,29


In [8]:
df_count_FP = df_merge_all_len[df_merge_all_len['flag'] == 'FP'].groupby(['tool','threshold','strain','len','n_true'],as_index=False)['flag'].count()
df_count_FP = df_count_FP.rename(columns={"flag": "nFP"})
df_count_FP.shape
df_count_FP.head()



,tool,threshold,strain,len,n_true,nFP
0,BioGraph*,10,AKR_J,100-500,234,153
1,BioGraph*,10,AKR_J,1000+,140,77
2,BioGraph*,10,AKR_J,50-100,37,132
3,BioGraph*,10,AKR_J,500-1000,83,27
4,BioGraph*,10,A_J,100-500,249,162


In [9]:
df_count_TN = df_merge_all_len[df_merge_all_len['flag'] == 'TN'].groupby(['tool','threshold','strain','len','n_true'],as_index=False)['flag'].count()
df_count_TN = df_count_TN.rename(columns={"flag": "nTN"})
df_count_TN.shape
df_count_TN.head()

,tool,threshold,strain,len,n_true,nTN
0,BioGraph*,10,AKR_J,1000+,140,205
1,BioGraph*,10,AKR_J,500-1000,83,1
2,BioGraph*,10,A_J,1000+,158,220
3,BioGraph*,10,BALB_CJ,1000+,157,235
4,BioGraph*,10,BALB_CJ,500-1000,77,2


In [10]:
#for s in strain:
    #for t in tools:
        #for th in [10,100,1000,10000]:
            #for l in ["50-100","100-500","500-1000","1000+"]:
                #if not (((df_count_TP['tool'] == t) & (df_count_TP['strain'] == s) & (df_count_TP['threshold'] == th) & (df_count_TP['len'] == l)).any()):
                    #df_count_TP = df_count_TP.append({'tool' : t , 'strain' : s, 'len' : l,'threshold' : th,'nTP' : 0} , ignore_index=True)
#df_count_TP.shape
#df_count_TP.head()

In [ ]:
df_merged = pd.DataFrame(columns=['tool', 'threshold', 'strain', 'len','nTP', 'nFP','nTN'])

for s in strain:
    for t in tools:
        for th in [10,100,1000,10000]:
            for l in ["50-100","100-500","500-1000","1000+"]:
                #dfObj = dfObj.append({'User_ID': 25, 'UserName': 'Jack', 'Action': 'Login'}, ignore_index=True)
                if (((df_count_TP['tool'] == t) & (df_count_TP['strain'] == s) & (df_count_TP['threshold'] == th) & (df_count_TP['len'] == l)).any()):
                    tp = df_count_TP[(df_count_TP['tool'] == t) & (df_count_TP['strain'] == s) & (df_count_TP['threshold'] == th) & (df_count_TP['len'] == l)]
                    tp = tp['nTP']
                    tp = tp.values[0]
                else:
                    tp = 0
                    
                if (((df_count_FP['tool'] == t) & (df_count_FP['strain'] == s) & (df_count_FP['threshold'] == th) & (df_count_FP['len'] == l)).any()):
                    fp = df_count_FP[(df_count_FP['tool'] == t) & (df_count_FP['strain'] == s) & (df_count_FP['threshold'] == th) & (df_count_FP['len'] == l)]
                    fp = fp['nFP']
                    fp = fp.values[0]
                else:
                    fp = 0
                 
                if (((df_count_TN['tool'] == t) & (df_count_TN['strain'] == s) & (df_count_TN['threshold'] == th) & (df_count_TN['len'] == l)).any()):
                    tn = df_count_TN[(df_count_TN['tool'] == t) & (df_count_TN['strain'] == s) & (df_count_TN['threshold'] == th) & (df_count_TN['len'] == l)]
                    tn = tn['nTN']
                    tn = tn.values[0]
                else: 
                    tn = 0
                
                #print(tp.values[0])
                df_merged = df_merged.append({'tool':t, 'threshold':th, 'strain':s, 'len':l,'nTP':tp, 'nFP':fp,'nTN':tn},ignore_index=True)
#df_count_TP.shape


In [ ]:
df_merged

In [ ]:
df_merged["threshold"] = pd.to_numeric(df_merged["threshold"])

In [ ]:
df_merge_all_len.dtypes

In [ ]:
df_merged_all=pd.merge(df_merged, df_merge_all_len,on=['strain','len','tool','threshold'])
del df_merged_all['length']
del df_merged_all['flag']
del df_merged_all['position']
df_merged_all

In [ ]:
df_merged_all['nFN']=df_merged_all['n_true']-df_merged_all['nTP']
df_merged_all

In [ ]:
df_merged_all = df_merged_all.astype({'nTP': 'float64', 'nFP': 'float64', 'nTN': 'float64', 'nFN': 'float64'})

df_merged_all['sensitivity']=df_merged_all['nTP']/df_merged_all['n_true']
df_merged_all['precision']=df_merged_all['nTP']/(df_merged_all['nTP'] + df_merged_all['nFP'])                                                                                    
df_merged_all['specificity']=df_merged_all['nTN']/(df_merged_all['nTN']+df_merged_all['nFP'])
df_merged_all['f-score']=2*(df_merged_all['sensitivity']*df_merged_all['precision'])/(df_merged_all['sensitivity']+df_merged_all['precision']+0.00000001)

df_merged_all = df_merged_all.astype({'nTP':'int64', 'nFP':'int64', 'nTN':'int64'})
df_merged_all.loc[np.isnan(df_merged_all['precision']), 'precision'] = 0

df_merged_all.tail()



In [ ]:
df_merged_all.to_csv('../Data/analysis_files/df_merged_all.csv')